If Dockerfiles have not been modified, connect to the Jupyter server with ```http://localhost:8006/tree?token=encode-panos```  

This notebook describes a pipeline to encode panoramas as described in Naik et al (2014).  
It takes:
- a base image as a ```.jpg```;
- its GIST descriptor as a ```.csv```;
- its textonmap as a ```.png```;
- its segmentations by ground, sky, vertical, and porous as ```.pgm```s; and  
returns a 5944×1 matrix as a ```.json``` encoding the image information.  

More specifically:
1. Each segmentation is converted into a 256×512 matrix of weights;
2. The segmentation weights are used to weight the histograms of the textonmap and base image, yielding 4 (for each segmentation) 8×8×8 histograms from the textonmap, and 4 14×4×4 histograms from the base image (an RGB to CIELAB colour conversion is first performed);
3. Flattening the 3000×1 GIST discriptor with 4 8×8×8 matrices and 4 14×4×4 matrices yields a 5944×1 matrix.  

> N. Naik, J. Philipoom, R. Raskar and C. Hidalgo, "Streetscore -- Predicting the Perceived Safety of One Million Streetscapes," 2014 IEEE Conference on Computer Vision and Pattern Recognition Workshops, Columbus, OH, USA, 2014, pp. 793-799, doi: 10.1109/CVPRW.2014.121.

In [1]:
target_dir = "data"
image_dir = "shunfu-panos"
gist_dir = "shunfu-gists"
textonmap_dir = "shunfu-textonmaps"
segmentation_dir = "shunfu-segmentations"
output_dir = "shunfu-panos-encoded"

In [2]:
import cv2
import numpy as np
import pandas as pd
from PIL import Image

import csv
import json
import os
from pathlib import Path

In [ ]:
images_path = []
for dirpath, dirnames, filenames in os.walk(os.path.join(target_dir, image_dir)):
    images_path.extend(filenames)
    break
images_id = {}
for image_path in images_path:
   images_id['.'.join(image_path.split('.')[:-1])] = os.path.join(target_dir, image_dir, image_path)
images_df = pd.DataFrame.from_dict(images_id, orient="index", columns=["fp"])

gists_path = []
for dirpath, dirnames, filenames in os.walk(os.path.join(target_dir, gist_dir)):
    gists_path.extend(filenames)
    break
gists_id = {}
for gist_path in gists_path:
   gists_id['.'.join(gist_path.split('.')[:-1])] = os.path.join(target_dir, gist_dir, gist_path)
gists_df = pd.DataFrame.from_dict(gists_id, orient="index", columns=["fp"])

textonmaps_path = []
for dirpath, dirnames, filenames in os.walk(os.path.join(target_dir, textonmap_dir)):
    textonmaps_path.extend(filenames)
    break
textonmaps_id = {}
for textonmap_path in textonmaps_path:
   textonmaps_id['.'.join(textonmap_path.split('.')[:-1])] = os.path.join(target_dir, textonmap_dir, textonmap_path)
textonmaps_df = pd.DataFrame.from_dict(textonmaps_id, orient="index", columns=["fp"])

segmentations_path = []
for dirpath, dirnames, filenames in os.walk(os.path.join(target_dir, segmentation_dir)):
    segmentations_path.extend(filenames)
    break
grounds_id = {}
verticals_id = {}
porouss_id = {}
skys_id = {}
for segmentation_path in segmentations_path:
    segmentation_type = segmentation_path.split('.')[1]
    if segmentation_type == "v000":
        grounds_id['.'.join(segmentation_path.split('.')[:-2])] = os.path.join(target_dir, segmentation_dir, segmentation_path)
    elif segmentation_type == "v090":
        verticals_id['.'.join(segmentation_path.split('.')[:-2])] = os.path.join(target_dir, segmentation_dir, segmentation_path)
    elif segmentation_type == "hpor":
        porouss_id['.'.join(segmentation_path.split('.')[:-2])] = os.path.join(target_dir, segmentation_dir, segmentation_path)
    elif segmentation_type == "vsky":
        skys_id['.'.join(segmentation_path.split('.')[:-2])] = os.path.join(target_dir, segmentation_dir, segmentation_path)
grounds_df = pd.DataFrame.from_dict(grounds_id, orient="index", columns=["fp"])
verticals_df = pd.DataFrame.from_dict(verticals_id, orient="index", columns=["fp"])
porouss_df = pd.DataFrame.from_dict(porouss_id, orient="index", columns=["fp"])
skys_df = pd.DataFrame.from_dict(skys_id, orient="index", columns=["fp"])

ids_df = images_df.join(
    gists_df, how="inner", lsuffix="_images", rsuffix="_gists").join(
        textonmaps_df, how="inner").join(
            grounds_df, how="inner", lsuffix="_textonmaps", rsuffix="_grounds").join(
                verticals_df, how="inner").join(
                    porouss_df, how="inner", lsuffix="_verticals", rsuffix="_porouss").join(
                        skys_df, how="inner").rename(columns={"fp": "fp_skys"})

In [ ]:
Path(os.path.join(target_dir, output_dir)).mkdir(parents=True, exist_ok=True)
count = 0
for index, row in ids_df.iterrows():
    try:
        image = cv2.imread(row["fp_images"], cv2.IMREAD_COLOR)
        gist = []
        with open(row["fp_gists"], 'r') as fp:
            csv_reader = csv.reader(fp)
            for line in csv_reader:
                if line:
                    gist.append(float(line[0]))
        area = sum(gist)
        gist = [x / area for x in gist]
        textonmap = cv2.imread(row["fp_textonmaps"], cv2.IMREAD_COLOR)
        ground = cv2.imread(row["fp_grounds"], cv2.IMREAD_GRAYSCALE)
        vertical = cv2.imread(row["fp_verticals"], cv2.IMREAD_GRAYSCALE)
        porous = cv2.imread(row["fp_porouss"], cv2.IMREAD_GRAYSCALE)
        sky = cv2.imread(row["fp_skys"], cv2.IMREAD_GRAYSCALE)
    except Exception as e:
        print(e)
        continue

    image = cv2.cvtColor(cv2.resize(image, (512, 256)), cv2.COLOR_BGR2LAB)
    textonmap = cv2.cvtColor(cv2.resize(textonmap, (512, 256)), cv2.COLOR_BGR2RGB)
    ground = cv2.resize(ground, (512, 256)).astype(float) / 255 * 100
    vertical = cv2.resize(vertical, (512, 256)).astype(float) / 255 * 100
    porous = cv2.resize(porous, (512, 256)).astype(float) / 255 * 100
    sky = cv2.resize(sky, (512, 256)).astype(float) / 255 * 100

    ground_texton_histogram, e = np.histogramdd(textonmap.reshape(256 * 512, 3), bins=[8, 8, 8], range=[(0, 256), (0, 256), (0, 256)], density=True, weights=ground.reshape(256 * 512))
    vertical_texton_histogram, e = np.histogramdd(textonmap.reshape(256 * 512, 3), bins=[8, 8, 8], range=[(0, 256), (0, 256), (0, 256)], density=True, weights=vertical.reshape(256 * 512))
    porous_texton_histogram, e = np.histogramdd(textonmap.reshape(256 * 512, 3), bins=[8, 8, 8], range=[(0, 256), (0, 256), (0, 256)], density=True, weights=porous.reshape(256 * 512))
    sky_texton_histogram, e = np.histogramdd(textonmap.reshape(256 * 512, 3), bins=[8, 8, 8], range=[(0, 256), (0, 256), (0, 256)], density=True, weights=sky.reshape(256 * 512))
    ground_colour_histogram, e = np.histogramdd(image.reshape(256 * 512, 3), bins=[14, 4, 4], range=[(0, 256), (0, 256), (0, 256)], density=True, weights=ground.reshape(256 * 512))
    vertical_colour_histogram, e = np.histogramdd(image.reshape(256 * 512, 3), bins=[14, 4, 4], range=[(0, 256), (0, 256), (0, 256)], density=True, weights=vertical.reshape(256 * 512))
    porous_colour_histogram, e = np.histogramdd(image.reshape(256 * 512, 3), bins=[14, 4, 4], range=[(0, 256), (0, 256), (0, 256)], density=True, weights=porous.reshape(256 * 512))
    sky_colour_histogram, e = np.histogramdd(image.reshape(256 * 512, 3), bins=[14, 4, 4], range=[(0, 256), (0, 256), (0, 256)], density=True, weights=sky.reshape(256 * 512))

    image_embedding = np.array(ground_texton_histogram).flatten().tolist()\
            + np.array(vertical_texton_histogram).flatten().tolist()\
            + np.array(porous_texton_histogram).flatten().tolist()\
            + np.array(sky_texton_histogram).flatten().tolist()\
        + gist\
        + np.array(ground_colour_histogram).flatten().tolist()\
            + np.array(vertical_colour_histogram).flatten().tolist()\
            + np.array(porous_colour_histogram).flatten().tolist()\
            + np.array(sky_colour_histogram).flatten().tolist()

    with open(os.path.join(target_dir, output_dir, index + ".json"), 'w') as fp:
        json.dump(image_embedding, fp)

    count += 1
    if not count % 50:
        print(count)